سروش تابش ۹۸۱۰۰۳۷۸
# پرسش ۲
برای حل این مسئله، ابتدا به صورت کمترین مربعات مدل می‌کنیم؛ مقادیر $I_i$
را در یک بردار $\tilde I$
قرار می‌دهیم و مقادیر $a_{ij}$
را نیز در یک ماتریس $A$
قرار می‌دهیم و مسئله به فرم بهینه‌سازی $\underset{x}{\min}{\lVert \tilde I - I\rVert_2}$
یا به عبارت دیگر، $\underset{p}{\min}{\lVert A p - I\rVert_2}$
می‌شود که $p$
بردار توان لامپ‌هاست.
اما شرط دیگری که در مسئله وجود دارد، محدودیت روی متغیر توان است که باید بین صفر و یک باشد. پس داریم:

$$p = \underset{0 \le p_i \le 1}{\text{argmin}}{\lVert A p - I\rVert_2^2}$$

برای حل این مسئله به دو روش می‌توانیم اقدام کنیم.    
روش اول، $\text{gradient descent}$
است که در جهت منفی گرادیان خطا که برابر $A^T(b-Ap)$
است حرکت می‌کنیم و زمانی که یک متغیر در مرز قرار دارد و بردار تغییر به سمت خارج فضای شدنی‌ست، تغییری در آن ایجاد نمی‌کنیم. از آنجایی که بهینه‌سازی درجه دو است، پس یک نقطه کمینه داریم و مشکل قرار گرفتن در کمینه‌های محلی را نداریم.    
روش دوم، که میخواهیم آن را پیاده کنیم، استفاده از الگوریتم     
[Bounded-Variable Least-Squares](https://www.stat.berkeley.edu/~stark/Preprints/bvls.pdf)    
است که در آن، به صورت مرحله به مرحله متغیر هایی که در مرز نیستند و یا روی مرز با گرادیان به سمت خارج هستند(این دو دسته از متغیر‌ها را متغیر آزاد می‌نامیم) را بهینه میکنیم و بقیه متغیر‌ها را ثابت نگه می‌داریم. با این‌کار در هر مرحله خطا اکیدن کاهش پیدا می‌کند و با سرعت بیشتری نسبت به روش قبل همگرا می‌شود. برای بهینه‌سازی متغیر‌های آزاد در هر مرحله، ابتدا با روش کمترین مربعات عادی، فقط روی متغیر‌های آزاد حل می‌کنیم و دیگر متغیرها را ثابت می‌گیریم؛ برای ثابت نگه داشتن دیگر متغیر‌ها، هر کدام را در ستون متناظرش در $A$
ضرب می‌کنیم، از $b$
کم میکنیم و آن را حذف می‌کنیم سپس متغیر‌های آزاد را با کمترین مربعات خطی عادی بهینه می‌کنیم و بردار به دست آمده را $z$
می نامیم. حال برای آپدیت کردن $p$،
درایه‌های متغیر‌های آزاد را در راستای $z-p$
حرکت می‌دهیم تا به $z$
یا به مرز برسند.


In [1]:
using LinearAlgebra, DelimitedFiles

data = readdlm("./data/Q2/data1.txt")
l = data[1:70, :]
v = data[71:121, :]
intensity = data[122, 1]

1.747845484

In [2]:
mids = (v+circshift(v, (-1, 0)))[1:end-1, :] ./ 2
normals = (circshift(v, (-1, 0))-v)[1:end-1, :] * [0 1; -1 0]
A = zeros((50, 70))
b = ones(50) .* intensity

for i = 1:50
    for j = 1:70
        d = l[j, :] - mids[i, :]
        A[i, j] = (normals[i, :] ⋅ d) / (norm(d) * norm(normals[i, :]))
        A[i, j] = max(A[i, j], 0) / (norm(d)^2)
    end
end

In [3]:
function BVLS(A, b, l, u, max_iter = 2 * size(A)[2], eps = 1e-12)
    m, n = size(A)
    F = Set([])
    U = Set([])
    L = Set(1:n)
    x = ones(n) * l
    iter = 0
    ign = []
    while length(F) < n && iter < max_iter
        iter += 1

        # Kuhn-Tucker convergence test
        w = A' * (b - A * x)
        if all(w[collect(L)] .<= 0) &&
           all(w[collect(U)] .>= 0) &&
           all((w[collect(F)] .< eps) .& (w[collect(F)] .> -eps))
            break
        end

        # Find critical variables which are not ignored
        T = setdiff(union(U, L), ign)
        if length(T) == 0
            break
        end

        # Extract critical variable with highest outward gradient
        sg = zeros(n)
        sg[collect(L)] .= 1
        sg[collect(U)] .= -1
        ts = collect(T)[argmax((t -> sg[t] * w[t]).(collect(T)))]
        pop!(L, ts, nothing)
        pop!(U, ts, nothing)
        push!(F, ts)

        # least squares on free variables
        B = collect(union(U, L))
        bp = vec(b - A[:, B] * x[B])
        Ap = A[:, collect(F)]
        z = Ap \ bp

        # if all free variables have optimal values inside the feasible set, we're done
        if all(z .> l) && all(z .< u)
            x[collect(F)] .= z
            continue
        end

        # limit solution to the bounds
        α = Inf
        for jp = 1:length(z)
            j = collect(F)[jp]
            if z[jp] > u || z[jp] < l
                if z[jp] > x[j]
                    a = (u - x[j]) / (z[jp] - x[j])
                else
                    a = (l - x[j]) / (z[jp] - x[j])
                end
                α = min(α, a)
            end
        end

        # if bounding is too small, ignore this new free variable
        if α < eps
            push!(ign, ts)
        else
            ign = []
        end

        # update free and critical variables
        Fnew = copy(F)
        for jp = 1:length(z)
            j = collect(F)[jp]
            x[j] += α * (z[jp] - x[j])
            if x[j] >= u - eps
                pop!(Fnew, j)
                push!(U, j)
            elseif x[j] <= l + eps
                pop!(Fnew, j)
                push!(L, j)
            end
        end
        F = Fnew
    end
    return x
end


p = BVLS(A, b, 0.0, 1.0)

70-element Array{Float64,1}:
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 0.9999999999999999
 1.0
 ⋮
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 0.9999999999999999
 1.0
 1.0

In [4]:
error = norm(A * p - b) / 50

0.2311439539605226